# Lab 4. Energy Agent Collaborator

## Introduction

In previous notebook we have shown you how to create multi-agent collaborator feature on Amazon Bedrock.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. 

In this notebook we will interact with our supervisor agent created in the previous lab.

The following represents the complete architecture of multi-agent collaborator.

![Architecture](img/energy_manager_agent.png)

On this lab you will execute a set of tests, using detailed trace, to check the chain-of-thought reasoning that is happening on the agents.

**[Chain-of-thought reasoning](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-prompt-engineering.html)** is a prompt engineering technique that give step-by-step reasoning on how an answer is derived based on your prompt.


With Amazon Bedrock Agents you can use [traces](https://docs.aws.amazon.com/bedrock/latest/userguide/trace-events.html) to understand the step-by-step reasoning process of your agent. With the multi-agent collaboration feature, your supervisor trace also provides the reasoning details of its sub-agents

## Setup

Make sure that your boto3 version is the latest one.

If not, return no [notebook 1](../1-energy-forecast/1_forecasting_agent.ipynb) and run Setup block again.

In [1]:
!pip freeze | grep boto3

boto3==1.40.55


## Invoking multi-agent Collaborator

Let's now invoke our supervisor agent with different queries to trigger the different sub-agents invocation. Before doing that, we're adding `bedrock_agent_helper.py` on Python path, so the file can be recognized and invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:

- `invoke`: invoking an agent with a query

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

curr_month = datetime.now()

In [3]:
import sys
import uuid
import time

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
agents = AgentsForAmazonBedrock()

Before continuing, let's recover the information from our supervisor agent and it's sub-agents from our environment.

In [4]:
%store -r
print("Supervisor agent name:", energy_agent_name, "id:", energy_agent_id)
print("Forecast agent name:", forecast_agent_name, "id:", forecast_agent_id, "alias_id:", forecast_agent_alias_id)
print("Solar Panel agent name:", solar_agent_name, "id:", solar_agent_id, "alias_id:", solar_agent_alias_id)
print("Peak Management agent name:", peak_agent_name, "id:", peak_agent_id, "alias_id:", peak_agent_alias_id)

Supervisor agent name: energy-agent-7eb12cba id: 4SWXQOA0GF
Forecast agent name: forecast-7eb12cba id: EGYPPNBQRV alias_id: I1KJYFXMQJ
Solar Panel agent name: solar-p-7eb12cba id: QVU1TYDKDP alias_id: 20BGU5GXVZ
Peak Management agent name: peak-agent-7eb12cba id: EDIE8XHWJV alias_id: QP3T1GZKEM


We can now set a dictionary with agents names for better visualization of the traces

In [5]:
multi_agent_names = {
    f"{forecast_agent_id}/{forecast_agent_alias_id}": forecast_agent_name,
    f"{solar_agent_id}/{solar_agent_alias_id}": solar_agent_name,
    f"{peak_agent_id}/{peak_agent_alias_id}": peak_agent_name,
    f"{energy_agent_id}/{energy_agent_alias_id}": energy_agent_name
}

On following section, you will invoke sub-agents but by multi-agent collaborator.

### Lab 1 - Forecasting Agent
First let's query our agent with prompts requiring the forecasting agent

In [6]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted energy consumption? How does it compare to my current usage? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

invokeAgent API request ID: c872a882-8bc6-44d8-98e8-904d3ccc24d1
invokeAgent API session ID: 0910acc2-ac38-11f0-a1a4-16ffda0c90af
---- Step 1 ----
Classifying request to immediately route to one collaborator if possible.
Routing classifier chose collaborator: 'ForecastCoordinationAgent'
Routing classifier took 0.6s, using 399 tokens (in: 383, out: 16).

---- Step 1.1 [using sub-agent name:forecast-7eb12cba, id:EGYPPNBQRV/I1KJYFXMQJ] ----
Took 4.9s, using 2498 tokens (in: 2284, out: 214) to complete prior action, observe, orchestrate.
To get the forecasted consumption for the next 3 months, I will call the forecast_consumption_actions::get_forecasted_consumption function with the provided customer ID. To get the current usage statistics, I will call the forecast_consumption_actions::get_consumption_statistics function.
Using tool: get_forecasted_consumption with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '1'}]

--tool outputs:
[{'customer_id': '1', 'day': '2025/11

In [7]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "can you give me my past energy consumption? What is my average spending on summer months? My customer id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

future_2m = curr_month + relativedelta(months=2)
future_2m_formatted = future_2m.strftime("%Y/%m")

response = agents.invoke(
    f"Can you update my forecast for month {future_2m_formatted}? I will be travelling and my estimate will be 50. My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you give me my forecasted energy consumption month by month? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 2 - Solar Panel Agent

In [8]:
%%time
session_id:str = str(uuid.uuid1())
response = agents.invoke(
    "how can I check if my Sunpower double-X solar panel eletrical consumption is compliant with energy rules?", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

invokeAgent API request ID: f5a836f5-5e53-4ed6-87d9-4df0bd54626e
invokeAgent API session ID: 4d8ed536-ac38-11f0-a1a4-16ffda0c90af
---- Step 1 ----
Classifying request to immediately route to one collaborator if possible.
Routing classifier chose collaborator: 'SolarSupportManagementAgent'
Routing classifier took 0.7s, using 402 tokens (in: 385, out: 17).

---- Step 1.1 [using sub-agent name:solar-p-7eb12cba, id:QVU1TYDKDP/20BGU5GXVZ] ----
Took 5.4s, using 1702 tokens (in: 1512, out: 190) to complete prior action, observe, orchestrate.
To check if your SunPower double-X solar panel electrical consumption is compliant with energy rules, I will first need to search the knowledge base for relevant information on:
1. SunPower double-X solar panel specifications
2. Electrical consumption standards and regulations for solar panels
3. How to verify compliance of solar panel electrical consumption
---- Step 1.2 [using sub-agent name:solar-p-7eb12cba, id:QVU1TYDKDP/20BGU5GXVZ] ----
Took 4.8s, us

In [9]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can I get all tickets that I have? My customer id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 3 - Peak Load Manager Agent

In [10]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "What's causing my peak load? My id is 2.", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

invokeAgent API request ID: 7db57f16-51a9-464d-8702-e77ff39617ae
invokeAgent API session ID: 7a9a15b8-ac38-11f0-a1a4-16ffda0c90af
---- Step 1 ----
Classifying request to immediately route to one collaborator if possible.
Routing classifier chose collaborator: 'PeakLoadOptimizationAgent'
Routing classifier took 0.6s, using 379 tokens (in: 361, out: 18).

---- Step 1.1 [using sub-agent name:peak-agent-7eb12cba, id:EDIE8XHWJV/QP3T1GZKEM] ----
Took 4.2s, using 1578 tokens (in: 1422, out: 156) to complete prior action, observe, orchestrate.
To answer this question, we need to detect the consumption peak for the customer and then identify the non-essential processes causing the peaks. Let's use the available functions to gather this information.
Using tool: detect_peak with these inputs:
[{'name': 'customer_id', 'type': 'string', 'value': '2'}]

--tool outputs:
[{'peak': 'True', 'quota': '100', 'item_id': '5', 'essential': 'False', 'customer_id': '2', 'used': '250', 'item_desc': 'gardening e

Exception: ('Unexpected exception: ', EventStreamError('An error occurred (throttlingException) when calling the InvokeAgent operation: Your request rate is too high. Reduce the frequency of requests. Check your Bedrock model invocation quotas to find the acceptable frequency.'))

In [11]:
time.sleep(60)

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Is it possible to optimize my consumption? My id is 1", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

#### Lab 4 - Multiple agents in parallel
Finally, let's submit a query which will result in our supervisor agent requiring responses from multiple agents in parallel. This shows the behaviour of a supervisor agent using supervisor mode rather than router mode as we have seen in the previous examples.

In [ ]:
%%time
session_id:str = str(uuid.uuid1())

response = agents.invoke(
    "Can you update my forecast for next month? I will be travelling and my estimate will be 90. My id is 1. Then tell me how can I check if my Sunpower double-X solar panel eletrical consumption is compliant with energy rules?", 
    energy_agent_id,
    session_id=session_id,
    enable_trace=True,
    multi_agent_names=multi_agent_names
)
print("====================")
print(response)

### Next Steps

Congratulations! You've completed the workshop! 

By now you have created 3 sub-agents and a supervisor agent. You have invoked the supervisor agent using prompts requiring multiple sub-agents.

For other multi-agent collaboration examples check the [Amazon Bedrock Agent Samples](https://github.com/awslabs/amazon-bedrock-agent-samples) repository 

Next, let's clean up our resources to avoid unexpected costs